In [4]:
# import os
# import torch
# import torchvision
# from torch.autograd import Variable
# import torch.nn as nn
# import torch.nn.functional as F

# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, utils
# import torch.optim as optim
# import torchvision.transforms as standard_transforms

# import numpy as np
# import glob
# import os
# import pickle
# import sys

In [1]:
import os
import os.path as osp
import sys
import time
import yaml
import cv2
import pprint
import traceback
import numpy as np

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.cuda.amp import autocast
from torch.nn.parallel import DistributedDataParallel as DDP
# from torch.utils.tensorboard import SummaryWriter
from torchvision import models

In [4]:
sys.path.insert(0, '/Users/narekgeghamyan/Classes/MLE_bootcamp/Capstone_Project')
from data.custom_dataset_data_loader import CustomDatasetDataLoader, sample_data
from model import U2NET

### Set-up Data

In [5]:
class parser(object):
    def __init__(self):
        self.name = "training_cloth_segm_u2net_exp1"  # Expriment name
        self.image_folder = "/Users/narekgeghamyan/local_data/capstone_data/imaterialist-fashion-2019-FGVC6/train"  # image folder path
        self.df_path = "/Users/narekgeghamyan/local_data/capstone_data/imaterialist-fashion-2019-FGVC6/train684.csv"  # label csv path
        self.distributed = False  # True for multi gpu training
        self.isTrain = True

        self.fine_width = 192 * 3
        self.fine_height = 192 * 3

        # Mean std params
        self.mean = 0.5
        self.std = 0.5

        self.batchSize = 2  # 12
        self.nThreads = 1 
        self.max_dataset_size = float("inf")

        self.serial_batches = False
        self.continue_train = True
        if self.continue_train:
            self.unet_checkpoint = "prev_checkpoints/cloth_segm_unet_surgery.pth"

        self.save_freq = 1000
        self.print_freq = 10
        self.image_log_freq = 100

        self.iter = 3
        self.lr = 0.0002
        self.clip_grad = 5

        self.logs_dir = osp.join("logs", self.name)
        self.save_dir = osp.join("results", self.name)

### Define Loss Function

In [8]:
#bce_loss = nn.BCELoss(reduction='mean')
loss_fxn = nn.CrossEntropyLoss(reduction='mean')

def loss_fuction(d0, d1, d2, d3, d4, d5, d6, labels_v):

	loss0 = loss_fxn(d0,labels_v)
	loss1 = loss_fxn(d1,labels_v)
	loss2 = loss_fxn(d2,labels_v)
	loss3 = loss_fxn(d3,labels_v)
	loss4 = loss_fxn(d4,labels_v)
	loss5 = loss_fxn(d5,labels_v)
	loss6 = loss_fxn(d6,labels_v)

	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data.item(),loss1.data.item(),loss2.data.item(),loss3.data.item(),loss4.data.item(),loss5.data.item(),loss6.data.item()))

	return loss0, loss

In [3]:
class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions 
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection 
        
        IoU = (intersection + smooth)/(union + smooth)
                
        return 1 - IoU

In [6]:
weights = np.array([1, 1.5, 1.5, 1.5], dtype=np.float32)
weights = torch.from_numpy(weights)
loss_IoU = IoULoss() 

### Define model

In [9]:
# define the net
model_name = 'u2net'
if(model_name=='u2net'):
#     net = U2NET(3, 1)
    net = U2NET(in_ch=3, out_ch=4)

if torch.cuda.is_available():
    net.cuda()
    print('Cuda available..')

### Define optimizer

In [10]:
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

### Train Model

In [11]:
epoch_num = 3
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0
save_frq = 3 # save every 2 iterations  

In [14]:
def training_loop(opt):
    
    custom_dataloader = CustomDatasetDataLoader()
    custom_dataloader.initialize(opt)
    loader = custom_dataloader.get_loader()
    
    device = torch.device("cuda:0")
    local_rank = 0
    
    net.train()
    get_data = sample_data(loader)
    for epoch in range(0, epoch_num):

        data_batch = next(get_data)
        image, label = data_batch
        image = Variable(image.to(device))
        label = label.type(torch.long)
        label = Variable(label.to(device))

        print(image)

        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

    #     inputs, labels = data['image'], data['label']

    #     inputs = inputs.type(torch.FloatTensor)
    #     labels = labels.type(torch.FloatTensor)

    #     # wrap them in Variable
    #     if torch.cuda.is_available():
    #         inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
    #                                                                                     requires_grad=False)
    #     else:
    #         inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6 = net(image)
        loss2, loss = loss_fuction(d0, d1, d2, d3, d4, d5, d6, label)

        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss #loss.data.item()
        running_tar_loss += loss2 #loss2.data.item()

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, loss2, loss

        print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
        epoch + 1, epoch_num, (i + 1) * batch_size_train, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

        if ite_num % save_frq == 0:

            torch.save(net.state_dict(), model_dir + model_name+"_bce_itr_%d_train_%3f_tar_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0

In [ ]:
opt = parser()
training_loop(opt)
print("Completed..............")
